# Top 3 Conditions by Current Temperature Interval



## Init

In [1]:
import pandas as pd
import os
import duckdb   

from dotenv import load_dotenv, find_dotenv
load_dotenv('/workspaces/patient_conditions_weather/src/.env')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/patient_conditions_weather/airflow/credentials/google_credentials.json'

### Load EMR Records

Currently loading from RAW and doing quick analytics features
This should be replaced by a raw - stg transformation

In [2]:
emr_path = f"gs://{os.getenv('EMR_RAW_BUCKET')}/{os.getenv('EMR_RAW_PATH')}"
encounters_path = emr_path +'/encounters.csv'
organizations_path = emr_path + '/organizations.csv'
conditions_path = emr_path + '/conditions.csv'

encounters = pd.read_csv(encounters_path, delimiter=",")
conditions = pd.read_csv(conditions_path, delimiter=",")
organizations = pd.read_csv(organizations_path, delimiter=",")

import re
rexp = r'\((.*?)\)'
conditions['condition_name'] = conditions['DESCRIPTION'].apply(lambda x: re.sub(r' \([^)]*\)', '', x))
conditions['condition_type'] = conditions['DESCRIPTION'].apply(lambda x: re.search(rexp, x).group(1) if re.search(rexp, x) else '')


### Load Daily Weather



In [3]:
weather_daily_data_path = f"gs://{os.getenv('WEATHER_BUCKET')}/{os.getenv('WEATHER_DATA_DAILY_PATH')}"
weather_daily_data = pd.read_parquet(weather_daily_data_path, engine="pyarrow")

### Register dckdb tables

In [8]:
# Create a DuckDB connection
con = duckdb.connect()
con.register('encounters', encounters)
con.register('conditions', conditions)
con.register('organizations', organizations)
con.register('weather_daily_data', weather_daily_data)

## Create table `diagnoses` 

In [9]:
starting_year = 1980
query = f"""
with events as (
    select 
        CAST(e.stop AS DATE) diagnosis_date,
        o.name, o.city,
        c.code,
        c.condition_name,
        c.condition_type,
        round(o.LAT,1) as latitude,
        round(o.LON,1) as longitude
    from encounters e
    inner join organizations o on e.ORGANIZATION = o.id
    inner join conditions c on e.id = c.ENCOUNTER
)
select *
from events
where extract('year' from diagnosis_date) >= {starting_year}
and condition_type in ('disorder', '')
order by 1
"""
diagnoses = con.execute(query).fetchdf()
con.register('diagnoses', diagnoses)

In [10]:
query = """
with weather as (
    select 
        cast(date as date) as weather_date,
        weather_code,
        temperature_2m_mean,	apparent_temperature_mean,	daylight_duration,	sunshine_duration,	latitude,	longitude
    from weather_daily_data     
)
select 
    d.diagnosis_date,
    d.code,
    d.condition_name,
    d.condition_type,
    w.daylight_duration,
    w.sunshine_duration,
    w.apparent_temperature_mean    
from diagnoses d 
inner join weather w on 
    w.latitude = d.latitude
    and w.longitude = d.longitude 
    and w.weather_date = d.diagnosis_date
"""
diagnoses_with_weather = con.execute(query).fetchdf()

diagnoses_with_weather

,diagnosis_date,CODE,condition_name,condition_type,daylight_duration,sunshine_duration,apparent_temperature_mean
0,1983-05-11,444814009,Viral sinusitis,disorder,52027.80,39295.40,3.7
1,1983-12-13,22298006,Myocardial Infarction,,32750.79,0.00,5.1
2,1983-04-01,40055000,Chronic sinusitis,disorder,45699.07,39281.17,-0.0
3,1983-04-05,239873007,Osteoarthritis of knee,,46387.60,32649.29,3.3
4,1983-05-24,53741008,Coronary Heart Disease,,53579.58,23740.59,15.3
...,...,...,...,...,...,...,...
265,1981-07-31,10509002,Acute bronchitis,disorder,52088.85,48600.56,22.2
266,1981-11-29,15777000,Prediabetes,,33809.44,21014.01,-6.0
267,1981-11-29,271737000,Anemia,disorder,33809.44,21014.01,-6.0
268,1981-10-09,59621000,Hypertension,,40976.31,34542.30,4.7


In [24]:
#diagnoses_with_weather['apparent_temperature_mean_q_bins'] = pd.qcut(diagnoses_with_weather['apparent_temperature_mean'], q=6)
diagnoses_with_weather['apparent_temperature_mean_bins'] = pd.cut(diagnoses_with_weather['apparent_temperature_mean'], bins=6, labels=None)
#diagnoses_with_weather['apparent_temperature_mean_bins'].unique()
diagnoses_with_weather

,diagnosis_date,CODE,condition_name,condition_type,daylight_duration,sunshine_duration,apparent_temperature_mean,apparent_temperature_mean_bins
0,1983-05-11,444814009,Viral sinusitis,disorder,52027.80,39295.40,3.7,"(3.5, 11.967]"
1,1983-12-13,22298006,Myocardial Infarction,,32750.79,0.00,5.1,"(3.5, 11.967]"
2,1983-04-01,40055000,Chronic sinusitis,disorder,45699.07,39281.17,-0.0,"(-4.967, 3.5]"
3,1983-04-05,239873007,Osteoarthritis of knee,,46387.60,32649.29,3.3,"(-4.967, 3.5]"
4,1983-05-24,53741008,Coronary Heart Disease,,53579.58,23740.59,15.3,"(11.967, 20.433]"
...,...,...,...,...,...,...,...,...
265,1981-07-31,10509002,Acute bronchitis,disorder,52088.85,48600.56,22.2,"(20.433, 28.9]"
266,1981-11-29,15777000,Prediabetes,,33809.44,21014.01,-6.0,"(-13.433, -4.967]"
267,1981-11-29,271737000,Anemia,disorder,33809.44,21014.01,-6.0,"(-13.433, -4.967]"
268,1981-10-09,59621000,Hypertension,,40976.31,34542.30,4.7,"(3.5, 11.967]"


In [18]:
diagnoses_with_weather

,diagnosis_date,CODE,condition_name,condition_type,daylight_duration,sunshine_duration,apparent_temperature_mean,apparent_temperature_mean_bins
0,1983-05-11,444814009,Viral sinusitis,disorder,52027.80,39295.40,3.7,"(3.5, 11.967]"
1,1983-12-13,22298006,Myocardial Infarction,,32750.79,0.00,5.1,"(3.5, 11.967]"
2,1983-04-01,40055000,Chronic sinusitis,disorder,45699.07,39281.17,-0.0,"(-4.967, 3.5]"
3,1983-04-05,239873007,Osteoarthritis of knee,,46387.60,32649.29,3.3,"(-4.967, 3.5]"
4,1983-05-24,53741008,Coronary Heart Disease,,53579.58,23740.59,15.3,"(11.967, 20.433]"
...,...,...,...,...,...,...,...,...
265,1981-07-31,10509002,Acute bronchitis,disorder,52088.85,48600.56,22.2,"(20.433, 28.9]"
266,1981-11-29,15777000,Prediabetes,,33809.44,21014.01,-6.0,"(-13.433, -4.967]"
267,1981-11-29,271737000,Anemia,disorder,33809.44,21014.01,-6.0,"(-13.433, -4.967]"
268,1981-10-09,59621000,Hypertension,,40976.31,34542.30,4.7,"(3.5, 11.967]"


In [26]:
diagnoses_with_weather.dtypes

diagnosis_date                    datetime64[us]
CODE                                       int64
condition_name                            object
condition_type                            object
daylight_duration                        float64
sunshine_duration                        float64
apparent_temperature_mean                float64
apparent_temperature_mean_bins          category
dtype: object

In [16]:
con.register('diagnoses_with_weather', diagnoses_with_weather)

RuntimeError: Unable to cast Python instance of type <class 'numpy.ndarray'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

**THE BINS ARE PROVIDED IN A FORMAT THAT DUCKDB REFUSES**

In [51]:


query = """
    select *
    from diagnoses_with_weather 
"""
a_df = con.execute(query).fetchdf()

RuntimeError: Unable to cast Python instance of type <class 'numpy.ndarray'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

In [45]:
query = """
    select *
    from diagnoses_with_weather 
"""
a_df = con.execute(query).fetchdf()
a_df

Error: Unable to cast Python instance of type <class 'numpy.ndarray'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)